# QF1: Laptop college1b

-------------------------------------------------------------------------------------------------

### Python functies en modules

* **sympy**: symbolische rekenmodule voor python. https://www.sympy.org/en/index.html. De mogelijkheden zijn eindeloos, maar een paar handige functies:
   - integrate(f(x), (x, x0, x1)) integreert een functie f(x) van x0 tot x1. Als grens kan je simpel oneindig aangeven met **oo**
   - simplify(expression) maakt de uitdrukking (meestal) simpeler
* **sympy.physics.quantum**: quantum module binnen sympy https://docs.sympy.org/latest/modules/physics/quantum/index.html. Enkele functies die we gebruik?en tijdens het laptopcollege zijn:
   - Wavefunction(psi(x),x) om een golffunctie te definieren
   - Wavefunction.expr:  geeft de functie psi(x)
   - Wavefunction.conjugate(): geeft de complex geconjugeerde van de golffunctie
   - DifferentialOperator(g(f(x)),f(x)): definieert een quantummechanische operator
   - qapply(operator\*wavefunction): laat een operator los op een golffunctie
*  **ipywidgets**: python module voor het maken van interactieve graphics https://ipywidgets.readthedocs.io/en/latest/.
   - interactive(f(i), i=(i0,i1)): roept functie aan met 'slider bar' die waarden tussen i0 en i1 kan aannemen. Handig om bijvoorbeeld in f(i) een plaatje te maken.
* **matplotlib**: plotting in python die jullie al eerder hebben gebruikt https://matplotlib.org/. Maar nu ook te gebruiken voor het maken van animaties.
   - animation.FuncAnimation(): maak een animatie van bijvoorbeeld een tijdsafhankelijke golffunctie.
   

In [ ]:
import sympy
from sympy import *
from sympy.plotting import plot
from sympy import Derivative, Function, Symbol
from sympy.physics.quantum.operator import DifferentialOperator
from sympy.physics.quantum.state import Wavefunction
from sympy.physics.quantum.qapply import qapply
import scipy.integrate as Nintegrate


%matplotlib inline
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np

## Opgave3: De Harmonische oscillator

In deze opdracht gaan we de harmonnische oscillator bestuderen, waarbij het doel is om de tijdsafhankelijke oplossingen van de Schrodinger vergeljking te bekijken. We gaan dus eigenfuncties vinden en gegeven begincondities de coefficienten uitrekenen die horen bij de stationaire oplossingen. Met behullp van een python animatie kan je kijken of je het gedrag van de harmonische oscillator kan begrijpen. Je kan zien wanneer de harmonische oscillator zich 'klassiek' gedraagt en wanneer het gedrag meer een 'quantum' signatuur heeft. 

------------------------------------------------------------------

#### Ladderoperatoren en Eigenfuncties

De Schrodinger vergelijking voor de harmonische oscillator (zie Griffiths 2.3) wordt gegeven door:

$$\imath \hbar \frac{\partial \Psi(x,t)}{\partial t} = -\frac{\hbar^2}{2 m} \frac{\partial^2\Psi(x,t)}{\partial x^2} + \frac{1}{2} m \omega^2 x^2 \Psi(x,t)$$ 

Voor de berekeningen tijdens het laptop college maken we ons leven gemakkelijk. We stellen: $\hbar=m=\omega=1$. De ladderoperatoren worden dan gegeven door:

$$a_{\pm} = \frac{1}{\sqrt{2}} \left( \mp i \hat{p} + \hat{x} \right) =  \frac{1}{\sqrt{2}} \left( \mp \frac{\partial}{\partial x} + x \right) $$

**3a)** Definieer de ladderoperatoren $a_{\pm}$ met behulp van de *DifferentialOperator* en *Derivative* uit sympy.

In [ ]:
# ladder operatoren
f= Function('psi')
x= Symbol('x')

a_plus = DifferentialOperator((-  Derivative(f(x),x) + x*f(x))/sympy.sqrt(2),f(x))
a_min  = DifferentialOperator((+  Derivative(f(x),x) + x*f(x))/sympy.sqrt(2),f(x))

**3b)** Definieer ook de Hamiltoniaan. (*Helaas is dat niet eenvoudig in sympy als je ladderoperatoren gebruikt: dus het beste kan je de Hamiltoniaan gebruiken zoals die hierboven in de Schrodinger vergelijking staat*)

In [ ]:
# hamiltoniaan
H = DifferentialOperator(-Derivative(f(x),(x,2))/2 + x**2/2 * f(x), f(x))

**3c)** Maak een functie die de n-de harmonische oscillator eigenfunctie teruggeeft. Dit zal dus een recursieve functie zijn waarbij je $a_+$ gebruikt om vanuit de grondtoestand de eigenfuncties met hogere energie te genereren. 

De golffunctie van de grondtoestand wordt gegeven door:

$$\psi_0(x) = \frac{1}{\pi^{1/4}} e^{-x^2/2}$$

In [ ]:
def generate_psi(n):
    #
    # genereer the n-de harmonische oscillator eigenfunctie
    #   
    
    #nn = Symbol('nn')
    
    psi=Wavefunction(sympy.exp(-x**2/2) / sympy.pi**(0.25), x)
    for i in range(0,n):
        # laat de a_plus operator werken op de golffunctie
        psi = qapply(a_plus*psi)
        # de factor 1/sqrt(i+1) is nodig om de normalisatie van de golffunctie te behouden
        g = psi.expr / sympy.sqrt(i+1)
        psi = Wavefunction(g,x)
    return psi

**3d)** Genereer de eigenfuncties met $n=5,10,12$ en bereken de energie-eigenwaarden. Kloppen de antwoorden met je verwachting?

In [ ]:
# Hint: gebruik qapply, de hamiltoniaan, en de functie generate_psi(n)....   


**3e)** Maak een functie om de orthogonaliteit van de oplossingen te verifieren: $\int_{-\infty}^{\infty} \psi_m(x)^* \psi_n(x) dx$. En test de functie op een paar paren van eigenfuncties.

In [ ]:
#check normalization
def orthonormality(i,j):
    value = integrate(generate_psi(i).expr * generate_psi(j).expr,(x,-oo,+oo))
    return value

In [ ]:
# Hint: voor het testen gebruik de functie hierboven.....

**3f)** Maak  een interactieve plot om de eigenfuncties weer te geven van $n=0 ... 50$. Verklaar waarom de eigenfuncties met hogere $n$ zich steeds verder uitstrekken tot grotere waarden van $x$.

In [ ]:
xmin = -10
xmax = +10

def wave_plot(m):
    title_string = '$\psi_{' +str(m)+'}(x)$'
    my_plot = plot(generate_psi(m).expr,(x,xmin,xmax), ylim=([-1.,1.]), title=title_string,ytitle='') 
    
interactive_plot = interactive(wave_plot, m=(0, 50))
interactive_plot

-----------------------------------------------------------------------

#### Oplossen van de tijdsafhankelijke Schrodinger vergelijking

Nu we de eigenfuncties $\psi_n(x)$ hebben, kunnen we de tijdsafhankelijke Schrodinger vergelijking oplossen. We gaan hiervoor te werk zoals in het college is besproken. De eerste stap is het definieren van een golffunctie, $\Psi(x,t=0)\equiv \Psi_0(x)$ als beginconditie.

Herhaal alle onderstaande berekeningen voor de volgende begincondities:
  - $\Psi_0(x)$ = Gauss + Gauss
  - $\Psi_0(x)$ = Gauss - Gauss
  - $\Psi_0(x)$ = Gauss

**3g)** Definieer en plot $\Psi(x,t=0)$.

In [ ]:
# Met de relatieve amplitude kan je kiezen of de tweede Gauss een relatief minteken of plusteken heeft. 
# Of je kan de tweede Gauss helemaal op 0 zetten.
relatieve_amplitude = +1

# positie, breedte en hoogte van de 1e Gauss
x0 = -3
sigma0 = 1.0
h0 = 1.0
# positie, breedte en hoogte van de 2e Gauss
x1 = +3
sigma1 = 1.0
h1 = 1.0

#
# definieer de golffunctie op t=0
#
Psi0 = Wavefunction(h0*exp(-(x-x0)**2/sigma0**2/2)+relatieve_amplitude*h1*exp(-(x+x0)**2/sigma1**2/2),x)
#
# niet vergeten te normaliseren
#
Psi0 = Psi0.normalize()
#
# ... en even te plotten
#
p=plot(Psi0.expr,(x,xmin,xmax))

**3h)** Bereken de coefficienten $c_n$ met onze 'standaard methode':
$$c_n = \int_{-\infty}^{+\infty} \psi_n^*(x) \Psi_0(x) dx$$

Als je een dubbel Gaussische verdeling hebt gedefinieerd voor $\Psi(x,t=0)$ leg dan uit wat opvalt aan $c_n$. 

In [ ]:
#
# aantal termen dat we willen hebben in onze oplossing....
#
nterm = 30

coef = []
energy = []
psi_arr = [] 

csum = 0
for i in range(nterm):
    psi_lambda = lambdify(x,generate_psi(i)(x),'scipy')
    psi_arr.append(psi_lambda)
    integrand = generate_psi(i).expr * Psi0.expr 
    integrand_lambda = lambdify(x,integrand,'scipy')
    c = Nintegrate.quad(integrand_lambda, -oo, +oo)[0]
    csum = csum + c*c
    coef.append(c)
    energy.append(i+1/2)
    print(i,' c(',i,')=',c,' E(',i,')=',i+1/2)

print('normalisatie csum = ',csum)

**3i)** Nu gaan we ons klaarmaken voor het plotten van de tijdsafhankelijke oplossing van de golffunctie. Daarvoor componeren we: $\sum c_n \psi_n(x) \exp(-\imath E_n t/\hbar)$.

In [ ]:
time = 0
def psi_anim(xx,t):
    value  = 0
    for i in range(nterm):
        value = value + coef[i]*psi_arr[i](xx)*np.exp(-1j*energy[i]*t)
    return value

**3j)** En animeer maar die golffunctie (duurt even, maar dan heb je ook wat). 
  - Verklaar wat je ziet. 
  - Leg uit of we hier te maken hebben met typisch klassiek of typisch quantum gedrag. Voor zowel alle $\Psi_0(x)$.
  - Alle $c_n \in \mathbf{R}$. Hoe komt het dat $\Psi(x,t)$ toch een imaginaire component heeft?
  - Waarom geldt $\Psi(x=0,t)=0$ als $\Psi(x,t=0)$ de asymmetrische dubbele Gauss is?

Is je opgevallen dat Gauss+Gauss en Gauss-Gauss op $t=0$ correspondeert met dezelfde kansverdeling, omdat $|\Psi(x,t=0)|^2$ in hetzelfde is. 
  - Kan je verklaren dat de tijdevolutie toch zo anders is? 

In [ ]:
#
# In this animation we show both Re(Psi(x,t)), Im(Psi(x,t), +-|Psi(x,t)|. 
#
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation

xlim = (-10,10)
# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure(figsize=(10,8))
ax = plt.axes(xlim=xlim, ylim=(-1, 1))
line_re, = ax.plot([], [], lw=1, linestyle=':', color='blue', label='Re($\Psi$)')
line_im, = ax.plot([], [], lw=1, linestyle='--', color='blue', label='Im($\Psi$)')
line_abs_pos, = ax.plot([], [], lw=2, linestyle='-', color='red', label='+-|$\Psi$|')
line_abs_neg, = ax.plot([], [], lw=2, linestyle='-', color='red')

ax.legend(prop=dict(size=12))
ax.set_xlabel('$x$',fontsize=18)
ax.set_ylabel(r'$\psi$',fontsize=18)


# initialization function: plot the background of each frame
def init():
    line_re.set_data([], [])
    line_im.set_data([], [])
    line_abs_pos.set_data([], [])
    line_abs_neg.set_data([], [])

    return line_re,line_im,line_abs_pos,line_abs_neg,

# animation function.  This is called sequentially
def animate(i):
    time = i*0.01
    xx = np.arange(xlim[0],xlim[1],0.01)
    val = psi_anim(xx,time)
    yre = val.real
    line_re.set_data(xx, yre)
    yim = val.imag
    line_im.set_data(xx, yim)
    
    line_abs_pos.set_data(xx,+abs(val))
    line_abs_neg.set_data(xx,-abs(val))

    return line_re,line_im,line_abs_pos,line_abs_neg,

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=500, interval=20, blit=True)

anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

#
# Hieronder zie je maar een enkele screenshot.... 
# Voer de notebook cell hieronder uit om de animatie te bekijken
#

In [ ]:
#
# this we use to display the animation in the jupyter notebook
#
from IPython.display import HTML

HTML(anim.to_html5_video())